# WK14

## Deep Dream

### Classification + Backpropagation

- Partially train CNN
- Get all partially activated classes at output and figure out which pixels activated them
- Change the pixels of the input image to fully activate those classes

#### Code:
- https://github.com/eriklindernoren/PyTorch-Deep-Dream/blob/master/deep_dream.py

#### Explanation:
- https://github.com/gordicaleksa/pytorch-deepdream
- https://github.com/ProGamerGov/neural-dream

In [ ]:
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/WK14/raw/main/WK14_utils.py

!wget -P ./data/image -q https://pytorch.org/tutorials/_static/img/neural-style/picasso.jpg
!wget -P ./data/image -q https://pytorch.org/tutorials/_static/img/neural-style/dancing.jpg

In [ ]:
import torch

from torch import nn, Tensor

from torchvision.models import resnet34, ResNet34_Weights
from torchvision.models import vgg19, VGG19_Weights
from torchvision.transforms import v2

from image_utils import make_image, open_image

from WK14_utils import count_parameters

In [ ]:
original_img = open_image("./data/image/dancing.jpg")
display(original_img)

In [ ]:
img_mean = Tensor([0.485, 0.456, 0.406])
img_std = Tensor([0.229, 0.224, 0.225])
img_unmean = img_mean.reshape(3,1,1)
img_unstd = img_std.reshape(3,1,1)

def unprocess_image(img_t, scale=1):
  img_t = img_t * img_unstd + img_unmean
  img_t = img_t.squeeze().permute(1, 2, 0)
  img_t = scale * img_t.clip(0.0, 1.0)
  return img_t

def clip_norm(device="cpu"):
  img_unmean = img_mean.reshape(3,1,1).to(device)
  img_unstd = img_std.reshape(3,1,1).to(device)
  def _clip_norm(img_t):
    return img_t.clip(-img_unmean / img_unstd, (1 - img_unmean) / img_unstd)
  return _clip_norm

In [ ]:
process_transform = v2.Compose([
  v2.Resize(512),
  v2.ToImage(),
  v2.ConvertImageDtype(torch.float),
  v2.Normalize(img_mean, img_std)
])

# Create Tensor for NN
original_t = process_transform(original_img).unsqueeze(0)
print(original_t.shape)

# Visualize NN Tensor
display(v2.ToPILImage()(original_t.squeeze()))

# Visualize un-normalized image
display(make_image(unprocess_image(original_t, 255)))

In [ ]:
mdevice = "cuda" if torch.cuda.is_available() else "cpu"

model_vgg = vgg19(weights=VGG19_Weights.DEFAULT).features.eval()
layers = list(model_vgg.children())
model = nn.Sequential(*list(model_vgg.children())[:28]).to(mdevice)

ClipNorm = clip_norm(mdevice)

print("full:", count_parameters(model_vgg), "dream:", count_parameters(model))
display(model)

lr = 1e-2

out = model(original_t.to(mdevice))
print(out.shape)

input_image = original_t.clone().to(mdevice)
input_image.requires_grad_(True)

In [ ]:
for e in range(32):
  model.zero_grad()
  out = model(input_image)
  loss = out.norm()
  loss.backward()

  avg_grad = input_image.grad.data.cpu().abs().mean()
  norm_lr = lr / avg_grad

  input_image.data += norm_lr * input_image.grad.data
  input_image.data = ClipNorm(input_image.data)
  input_image.grad.data.zero_()

  if e % 4 == 3:
    print(f"Epoch: {e} loss: {loss.item():.4f}")

In [ ]:
output_image = display(make_image(unprocess_image(input_image.to("cpu"), 255)))

## Possible Next Steps

- Implement octave scaling to add different levels of detail to images
- Use custom classifier trained on custom dataset to activates other types of shapes